
# About the data:

## The data is about hospital patient data.
## The problem statement is: the clinic has gotten several complaints regarding wait times.
 ____________
## -- The requirements are:
### Dose the patient type affect the waiting time?
### Is there a specific type of patient waiting a long time?
### Are we too busy?
### Do we have staffing issues?
### How much the patients wait before the doctor can see them?
### What type of staff do we need or where do we need them?
### What days of the week are affected?
### How can we fix it?

In [ ]:
!pip install openpyxl
!pip install pandas==1.3.5

In [ ]:
import pandas as pp
import numpy as nn
import plotly.express as px
import plotly.offline as py
template_style = "plotly_dark"
%matplotlib inline

In [ ]:
df = pp.read_excel('hospital_data_sampleee.xlsx')      #read the file 
df.head(2)

## EDA/CLEANING

In [ ]:
df.info()       #the information of the data

In [ ]:
df.shape

In [ ]:
df.duplicated().sum()           #Is there any duplicated values?

In [ ]:
df.isnull().sum().sort_values(ascending=False)          #is there any NULL values?

In [ ]:
df.nunique()

In [ ]:
df_copy = df.copy()         # always make sure u keep a copy

In [ ]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-","_" )       ## change the names forms

In [ ]:
df.head()

### let's look at every single columns

In [ ]:
df.date.value_counts().sort_index()

In [ ]:
df.medication_revenue.value_counts()

In [ ]:
df.lab_cost.value_counts()

In [ ]:
df.consultation_revenue.value_counts()

In [ ]:
df.doctor_type.value_counts()

In [ ]:
df.financial_class.value_counts()

In [ ]:
df.patient_type.value_counts()

In [ ]:
df.entry_time.value_counts()

In [ ]:
df.post_consultation_time.value_counts()

In [ ]:
df.completion_time.value_counts()

In [ ]:
df.patient_id.value_counts()

### (EDA)

In [ ]:
df.dtypes

### We don't have the data for patient time waiting, so let's add the timing by sub 'entry time' - 'completion time'

### We cant sub two dates.time so we need to change 'entry_time' , 'post_consultation_time' , 'completion_time' to_timedelta

In [ ]:
df.completion_time = pp.to_timedelta(df.completion_time.astype(str))
df.entry_time = pp.to_timedelta(df.entry_time.astype(str))
df.post_consultation_time = pp.to_timedelta(df.post_consultation_time.astype(str))

In [ ]:
df.dtypes

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.head(2)

#### just ignore the 0 days

### We just need the 'waiting_time', but ill add some more to make it easy for me

In [ ]:
df['waiting_time'] = df['completion_time'] - df['entry_time']  # new column for Calculator 'waiting_time'
###
df['waiting_ber_munets'] = df.waiting_time.dt.seconds / 60     # new column for Calculator 'waiting_ber_munets'
df['waiting_ber_munets'] = df['waiting_ber_munets'].round(0)
####
df['weekday'] = df.date.dt.strftime('%A')                      # new column for extract 'weekday' from 'date'
df['hours'] = df.entry_time.dt.components.hours                # new column for extract the  'hours' from 'entry_time'
df.head()

### The first thing they want to know is: Dose the patient type affect the waiting time? , And is there a specific type of patients waiting a long time?

In [ ]:
df.patient_type.value_counts()          #checking the values of 'patient_type'

In [ ]:
df.financial_class.value_counts()

### We just have one type of patient in the 'patient_type', but if we can use 'financial_class' I think we can answer the Q

#### And we will do the same for 'Weekday' later (the second Q) , so let's build a function for Groupby

In [ ]:

# Groupby as a function
def grouped_data(column_name):
    '''
    Groupby column and return DataFrame
    Input: Column Name
    '''
    df_tmp = df.groupby(column_name)[['waiting_ber_munets']].mean().round(0)
    df_tmp2 = df[column_name].value_counts()      #this for concat
    
    pivot_F = pp.concat([df_tmp, df_tmp2.rename('number_of_patient')],axis=1)
    pivot_F.reset_index(inplace=True)
    pivot_F = pivot_F.rename(columns={'index': column_name })
    return pivot_F
    
answer1 =  grouped_data('financial_class')
answer2 =  grouped_data('weekday')
print(answer1 , answer2)              #checking


In [ ]:
fig = px.pie(answer1, values='number_of_patient', 
             names= 'financial_class' , hole=0.6 , width=600,height=600 , 
             template=template_style ,
             hover_data=['waiting_ber_munets'], 
             labels={'waiting_ber_munets':'the waiting time per m'})
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()
# py.offline.plot(fig, filename='{}.html'.format(r"C:\" File Path "\\"),
#                 auto_open=False, image_width=1600,
#                         image_height=900)                     #import the plot to html file

## So the Q was: Dose the patient type affect the waiting time?

#### We can see that : 
#### - The 'INSURANCE' type of patient is (33.1%) of our patients and they are (9931 patients) and they waiting (44 min ) on average. 
#### - The 'PRIVATE' type of patient is (30.4%) of our patients and they are (9121 patients) and they waiting (40 min ) on average. 
#### - The 'CORPORATE' type of patient is (23.1%) of our patients and they are (6915 patients) and they wait (46 min ) on average. 
#### - The 'HMO' type of patient is (12.5%) of our patients and they are (3738 patients) and they waiting (46 min ) on average. 
#### - The 'MEDICARE' type of patient is (1%) of our patients and they are (293 patients) and they waiting (58 min ) on average. 

#### SO No the patient type does not give that big effect on the waiting time, BUT if you 'MEDICARE' type of patient your chance to wait long is higher.

_________________________________________

## Is there a specific type of patient waiting a long time?

#### Yes there is a specific type of patient waiting a long time, and that type is MEDICARE and they wait (58 min) on average.

___________________________________________________________________

## Are we too busy?
#### I'll do daily and hourly visualization to answer the Q, and build two heatmaps one for 'wait time' and the second for 'the number of patients, so lets make a function to make it easy

In [ ]:
def grouped_data2(column_name):
    if column_name == 'completion_time':                                            # if column_name == 'completion_time' do the following
        answer = pp.pivot_table(df, index='hours',                                  # make a pivot_table and aggfunc 'count'
                                columns=['weekday'], aggfunc= 'count')
        answer.drop(answer.iloc[: , 7:], axis=1, inplace=True)                      # Drop the columns [: , 7:]
    elif column_name == 'waiting_ber_munets':                                       # if column_name == 'waiting_ber_munets' do the following
        answer = pp.pivot_table(df, index='hours',                                  # make a pivot_table and aggfunc 'mean'
                                columns=['weekday'] , aggfunc= 'mean').round(1)  
    else:
        return(column_name + " is not in the columns")                              # if the giving value is else print this statment
    
    answer = answer[column_name]
    answer = answer.fillna(0)
    answer = answer[['Sunday', 'Monday', 'Tuesday',                                 # sorting
                  'Wednesday', 'Thursday', 
                  'Friday', 'Saturday']] 
    return answer
answer3 =  grouped_data2('completion_time')
answer4 =  grouped_data2('waiting_ber_munets')
print(answer3,answer4)      #checking

In [ ]:
fig3 = px.imshow(answer3,
                labels=dict(x="weekday", y="hours", color="number of paitant") , 
                aspect="auto", color_continuous_scale='tempo',
                template = template_style, title="Daily/hourly visualization",
                text_auto=True, width=700, height=700)
fig3.update_xaxes(side="top")
fig3.show()

#### I assumed that the highest number of patients = the longest wait time.
#### I noticed that there are times in the day when the number of patients is low, and these times are: (7 hour) and (13 hour) and (17 hour) and (21, and after)

In [ ]:
fig4 = px.imshow(answer4,
                labels=dict(x="weekday", y="hours",
                            color="the waiting time per min") , 
                            aspect="auto", color_continuous_scale='tempo',
                            template = template_style,
                            text_auto=True, width=700, height=700)
fig4.update_xaxes(side="top")
fig4.show()

### Yes, we are too busy in the morning period and at 13 hour
#### In the first heatMap I thoughts it will be less wait time for patients at 7, 13, 17, 21, 22, and 23, But here I found that the rushed time for waiting started at 7 on(Tuesday, Wednesday, Thursday, Saturday) and reach the peak at 9, and decrease after, and start increasing at 13 and decreased after, and increased at 17 Until 20 after that it's decreasing, that means we are short in staff, but im curious about the 7 hour, why the wait time is hghi?

In [ ]:
def morining(colmun_n):
    ddf = df[['entry_time' , 'post_consultation_time' , 'completion_time' , 'waiting_ber_munets']].sort_values(by= colmun_n ).head(10)
    return ddf

In [ ]:
morining('entry_time').head(2)

#### It seems that the patients enter after 7:50 so maybe the clinic(OUTPATIENT) Starts at 8:00 

In [ ]:
morining('post_consultation_time').head(2)

#### it seems that im right the(OUTPATIENT) started at 8:07

### I'll build another chart to make sure that we are having a staff issue and nothing else

In [ ]:
answer5 = grouped_data('hours')
link_size = [3,6]
fig = px.bar(x=answer5['hours'],
             y=answer5['waiting_ber_munets'],
             template= template_style,
             text_auto='.2s',
             labels={'x':'the hour',
                     'y':'the waiting time per m'}
             ).add_traces(
      px.line(answer5, x=answer5['hours'], text='number_of_patient',
             y=answer5['number_of_patient'],markers=True).update_traces(yaxis="y2",
             showlegend=True, line=dict(color = 'red', width=link_size[1]), name="number_of_patient").data)        
fig.update_layout(yaxis2={"side":"right", "overlaying":"y"})
fig.show()


#### Now we can see that (13 and 17) have an average wait time even though we have a lower amount of patients, so Yes it's a staffing issue

## So how can we fix this issue?
#### increasing our staffing at these hours (8 and 9,13 and 14,18) hours 

## What type of staff we need or where do we need them?

### Let's break this down, we have 4 types of timing in the data,
#### entry_time = entered the (OUTPATIENT)
#### post_consultation_time = when the doctor tells the patients to enter the clinic room
#### completion_time = when the patients exit the clinic room or the building
#### waiting_time = the entire wait time spend in the hospital
__________
we can extract more info from these like.
#### consultation_period = before entering to the doctor
#### process_period = talking to the doctor
#### consultation_perc = the % of time spend in consultation_period
#### process_perc = is the rest of the % of the consultation_perc time

In [ ]:
df['consultation_period'] = df.post_consultation_time - df.entry_time
df['consultation_period'] = df.consultation_period.dt.seconds / 60          
df['consultation_period'] = df['consultation_period'].round(2)
# consultation_period = the amount of time after entr the (OUTPATIENT) and before the doctor see the patient

df['process_period'] = df.completion_time - df.post_consultation_time
df['process_period'] = df.process_period.dt.seconds / 60                     
df['process_period'] = df['process_period'].round(2)
# process_period = the amount of time the patients talking to the doctort + exite the (OUTPATIENT)

df['consultation_perc'] = df.consultation_period / df.waiting_ber_munets
df['consultation_perc'] = df['consultation_perc'].round(2)
# consultation_perc = the % of time spend in consultation_period

df['process_perc'] = 1 - df.consultation_perc
# process_perc = is the rest of the % of the consultation_perc time
df.head(1)

# How much the patients wait before the doctor can see them?

In [ ]:
df.consultation_period.mean()

### The patients wait before the doctor can see them 38min in average

In [ ]:
answer6 = df[['consultation_perc', 'process_perc']].mean()
periods = df[[ 'consultation_period', 'process_period']].mean().round(0)
print(answer6)

In [ ]:
fig = px.pie(answer6 , names= answer6.index, 
             template=template_style, 
             values= answer6,
             hover_name= periods.index,
             color=periods,
             labels={'color':'the waiting time per m '
                     })
fig.update_traces(textposition='outside',
                  textinfo='percent+label')
fig.show()

#### soo 88% of our patient time is gone for consultation_period whats means our patients spend 88% (average 39 min)of their time waiting for the doctor and spend just 11% talking to the doctor.

### as we can see the time to wait to see the doctor is limited by the number of doctors.

### The next Q was, What days of week are affected?

In [ ]:
# Create Chart for the Daily
fig2 = px.bar(answer2,
             x='weekday',
             y='number_of_patient',
             color='waiting_ber_munets',
             labels={'waiting_ber_munets':'the waiting time per m'} ,
             color_continuous_scale=['green','yellow','red'],
             template = template_style,
             title = '<b>Daily visualization</b>')
# Display Plot
fig2.show()

##  To ansowe the Q, the affected day is Monday and Wednesday.
#### 'Monday' has a lot of patients(6982) and waiting time(49 min) (correlated)
#### 'Wednesday' has the average number of patients(4171) but high waiting time(47 min ) (not correlated)

# Summary: 
### There may be a possibility to add more medical staff during these times ((8 and 9,13 and 14,18) of the days), and focus on Monday and Wednesday.

# Actions:
### Determine if it makes financial sense to have an additional medical staffing at these times in summary section .

## The full project here : https://github.com/Abdulqader-Asiri/Portfolio_Projects